In [ ]:
Nicholas Chludzinski

In [ ]:
import cv2
import numpy as np

# Load YOLO
weights_path = '/Users/nicho/Downloads/YOLO/yolov4.weights'  
config_path = '/Users/nicho/Downloads/YOLO/yolov4.cfg'      
coco_names_path = '/Users/nicho/Downloads/YOLO/coco.names'

net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()

# Get the output layer names
output_layers_indices = net.getUnconnectedOutLayers()
output_layers = [layer_names[i - 1] for i in output_layers_indices]  # Adjusted here

# Load COCO class labels
with open(coco_names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize video capture on webcam
cap = cv2.VideoCapture(1)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    
    # Check if frame is captured
    if not ret:
        print("Error: Could not read frame.")
        break

    # Resize the frame
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    # Process outputs
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Get the bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw rectangles around detected objects
    for i in indices:
        box = boxes[i]
        (x, y, w, h) = box
        label = str(classes[class_ids[i]])
        
        # Draw rectangle and label for all detected objects
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Display the frame with detected objects
    cv2.imshow('Object Detection with YOLO', frame)

    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and destroy all windows
cap.release()
cv2.destroyAllWindows()
